In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import lfilter,correlate
import tensorflow.keras.layers as tfl
import tensorflow as tf
from keras.callbacks import ReduceLROnPlateau
from PIL import Image

2024-06-22 18:54:37.568408: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-22 18:54:37.568490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-22 18:54:37.693825: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def pre_emphasis(signal,factor=0.97):
    return np.append(signal[0], signal[1:] - factor * signal[:-1])

In [3]:
def inverse_filtering(signal, lpc_coeffs):
    return lfilter(np.concatenate(([1], -lpc_coeffs[1:])), [1], signal)

In [4]:
def compute_lpc(signal, order):
    autocorr = correlate(signal, signal, mode='full')
    autocorr = autocorr[len(signal)-1:len(signal)+order]
    
    a = np.concatenate(([1], -autocorr[1:order+1]))
    b = [1]
    
    lpc_coeffs = lfilter(b, a, signal)
    
    lpc_coeffs = np.concatenate(([1], -lpc_coeffs[1:order+1]))
    
    del autocorr,a,b
    
    return lpc_coeffs

In [5]:
def lpc_to_cepstrum(lpc_coeffs):
    return np.fft.ifft(np.log(np.abs(np.fft.fft(lpc_coeffs))))

In [6]:
import gc
from IPython.display import clear_output

def create_spectrograms(train_folder, create_folder, verbose=False, speakers=50, utterances=10, sr=7000, frame_length=2048, hop_length=512, lpc_order=16):
    spc_folder = os.path.join(create_folder, "spectrogram")
    vt_folder = os.path.join(create_folder, "vocal_tract")
    glot_folder = os.path.join(create_folder, "glottal")
    
    total_speaker = 0
    for speaker in os.listdir(train_folder):
        total_utterances = 0
        speaker_folder_spc = os.path.join(spc_folder, speaker)
        speaker_folder_vt = os.path.join(vt_folder, speaker)
        speaker_folder_glot = os.path.join(glot_folder, speaker)
        os.makedirs(speaker_folder_spc, exist_ok=True)
        os.makedirs(speaker_folder_vt, exist_ok=True)
        os.makedirs(speaker_folder_glot, exist_ok=True)
        
        for vidID in os.listdir(os.path.join(train_folder, speaker)):
            for file in os.listdir(os.path.join(train_folder, speaker, vidID)):
                if file.endswith(".wav"):
                    wav_file_path = os.path.join(train_folder, speaker, vidID, file)
                    
                    y, sr = librosa.load(wav_file_path, sr=sr)
                    
#                     y_preemphasized=pre_emphasis(y)
                    y_preemphasized=y
                    
                    frames = librosa.util.frame(y_preemphasized, frame_length=frame_length, hop_length=hop_length).T
                    del y_preemphasized
                    
                    window = np.hamming(frame_length)
                    frames_windowed = frames * window  
                    del frames
                    
                    lpc_coeffs = []
                    for frame in frames_windowed:
                        frame=np.array(frame)
                        coeff = compute_lpc(frame, lpc_order)
                        lpc_coeffs.append(coeff)
                        
                    glottal_waveforms = [inverse_filtering(frame, coeff) for frame, coeff in zip(frames_windowed, lpc_coeffs)]
                    glottal_waveforms_avg = np.mean(np.abs(np.array(glottal_waveforms)), axis=0)
                    del glottal_waveforms
                    
                    residuals = []
                    for i, coeff in enumerate(lpc_coeffs):
                        frame = frames_windowed[i]
                        residual = lfilter(coeff, [1.0], frame)
                        residuals.append(residual)
                        
                    residual_avg = np.mean(np.abs(np.array(residuals)), axis=0)
                    
                    del frames_windowed, lpc_coeffs, residuals
                    gc.collect()

                    try:
                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(y))
                        D = librosa.amplitude_to_db(amp, ref=np.max)
                        del y,amp
                        librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_spc, "spc"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D,ax,output_path
                        gc.collect()

                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(residual_avg))
                        D_residual = librosa.amplitude_to_db(amp, ref=np.max)
                        del amp
                        librosa.display.specshow(D_residual, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_vt, "vt"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D_residual,ax,output_path
                        gc.collect()

                        plt.figure(figsize=(5, 5))
                        ax = plt.axes()
                        ax.set_axis_off()
                        plt.set_cmap('hot')
                        amp=np.abs(librosa.stft(glottal_waveforms_avg))
                        D_glot = librosa.amplitude_to_db(amp, ref=np.max)
                        del amp
                        librosa.display.specshow(D_glot, sr=sr, x_axis='time', y_axis='log')
                        output_path = os.path.join(speaker_folder_glot, "glot"+str(total_utterances+1) + ".png")
                        plt.savefig(output_path, bbox_inches='tight', transparent=True, pad_inches=0.0)
                        plt.close('all')
                        del D_glot,ax,output_path
                        gc.collect()
                        
#                         clear_output()
                    
                    except OSError as e:
                        print(f"Error saving spectrogram for {wav_file_path}: {e}")
                    
                    del residual_avg, glottal_waveforms_avg
                    gc.collect()
    
                    total_utterances += 1
                    if total_utterances == utterances:
                        break
            
            if total_utterances == utterances:
                break
        
        total_speaker += 1
        if total_speaker != 0 and total_speaker % 1 == 0 and verbose:
            print(f"{total_speaker} speakers completed.\n")
        
        if total_speaker == speakers:
            break

In [7]:
raw_dataset_loc="/kaggle/input/voxceleb1train/wav"
save_loc="/kaggle/working/"
num_speakers=10
num_utterances=100

In [8]:
create_spectrograms(raw_dataset_loc,save_loc,True,num_speakers,num_utterances)

1 speakers completed.

2 speakers completed.

3 speakers completed.

4 speakers completed.

5 speakers completed.

6 speakers completed.

7 speakers completed.

8 speakers completed.

9 speakers completed.

10 speakers completed.



In [9]:
dataset_path_spc="/kaggle/working/spectrogram"
dataset_path_vt="/kaggle/working/vocal_tract"
dataset_path_glot="/kaggle/working/glottal"
folders=os.listdir(dataset_path_spc)

In [10]:
dict={}
i=0
for val in folders:
    dict[val]=i
    i=i+1

dict

{'id10719': 0,
 'id10484': 1,
 'id10480': 2,
 'id10036': 3,
 'id11123': 4,
 'id10061': 5,
 'id10459': 6,
 'id11079': 7,
 'id10116': 8,
 'id11250': 9}

In [11]:
dataset_path_spc="/kaggle/working/spectrogram"
dataset_path_vt="/kaggle/working/vocal_tract"
dataset_path_glot="/kaggle/working/glottal"
folders=os.listdir(dataset_path_spc)

# Initializing training and test dataset
X_train=[[],[],[]]
y_train=[[],[],[]]
X_test=[[],[],[]]
y_test=[[],[],[]]

# Split the dataset into training and test set.
num=np.random.rand(num_utterances*num_speakers)
mask=num<0.2
split=mask.astype(int)

for dirs in folders:
    i=0
    for img in os.listdir(os.path.join(dataset_path_spc,dirs)):
        image=Image.open(os.path.join(dataset_path_spc,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[0].append(tmp_array)
            y_train[0].append(dict[str(dirs)])
        else:
            X_test[0].append(tmp_array)
            y_test[0].append(dict[str(dirs)])
            
        i+=1
        
    i=0    
    for img in os.listdir(os.path.join(dataset_path_vt,dirs)):
        image=Image.open(os.path.join(dataset_path_vt,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[1].append(tmp_array)
            y_train[1].append(dict[str(dirs)])
        else:
            X_test[1].append(tmp_array)
            y_test[1].append(dict[str(dirs)])
            
        i+=1
            
    i=0
    for img in os.listdir(os.path.join(dataset_path_glot,dirs)):
        image=Image.open(os.path.join(dataset_path_glot,dirs,img))
        new_img=image.resize((200,200))
        tmp_array=np.array(new_img)/255.
        if split[i]==0:
            X_train[2].append(tmp_array)
            y_train[2].append(dict[str(dirs)])
        else:
            X_test[2].append(tmp_array)
            y_test[2].append(dict[str(dirs)])
        
        i+=1

In [12]:
def create_train_triplets(X_train, y_train, num_triplets):
    triplets = []
    labels = np.unique(y_train)
    
    for _ in range(num_triplets):
        anchor_label = np.random.choice(labels)
        negative_label = np.random.choice(labels[labels != anchor_label])
        
        anchor_indices = np.where(y_train == anchor_label)[0]
        positive_indices = np.where(y_train == anchor_label)[0]
        negative_indices = np.where(y_train == negative_label)[0]
        
        anchor = X_train[np.random.choice(anchor_indices)]
        positive = X_train[np.random.choice(positive_indices)]
        negative = X_train[np.random.choice(negative_indices)]
        
        triplets.append((anchor, positive, negative))
    
    return np.array(triplets)

In [13]:
def create_test_triplets(X_test, y_test, num_triplets):
    triplets = []
    range_test = len(y_test)
    idx = np.arange(range_test)
    
    for _ in range(num_triplets):
        label_1 = np.random.choice(idx)
        label_2 = np.random.choice(idx)
        
        anchor = X_test[label_1]
        inp = X_test[label_2]
        out = 1 if y_test[label_1] == y_test[label_2] else 0
        
        triplets.append((anchor, inp, out))
        
    return np.array(triplets, dtype=object)  

In [14]:
def convolutional_model(input_shape):
    input_img = tf.keras.Input(shape=input_shape)
    Z1=tfl.Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(input_img)
    A1=tfl.ReLU()(Z1)
    P1=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A1)
    Z2=tfl.Conv2D(filters=64,kernel_size=(2,2),strides=(1,1),padding='same')(P1)
    A2=tfl.ReLU()(Z2)
    P2=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A2)
    Z3=tfl.Conv2D(filters=128,kernel_size=(2,2),strides=(1,1),padding='same')(P2)
    A3=tfl.ReLU()(Z3)
    P3=tfl.MaxPool2D(pool_size=(4,4),padding='same')(A3)
    F=tfl.Flatten()(P3)
    D1=tfl.Dense(128)(F)
        
    model = tf.keras.Model(inputs=input_img, outputs=D1)
    return model

input_shape = (200, 200, 4)
base_model = convolutional_model(input_shape)

In [42]:
def triplet_loss(y_true, y_pred, margin=0.2):
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=-1)
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

In [43]:
anchor_input = tfl.Input(shape=input_shape, name='anchor_input')
positive_input = tfl.Input(shape=input_shape, name='positive_input')
negative_input = tfl.Input(shape=input_shape, name='negative_input')

encoded_anchor = base_model(anchor_input)
encoded_positive = base_model(positive_input)
encoded_negative = base_model(negative_input)

triplet_model_spc = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_spc.compile(optimizer='adam', loss=triplet_loss)

triplet_model_vt = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_vt.compile(optimizer='adam', loss=triplet_loss)

triplet_model_glot = tf.keras.Model(inputs=[anchor_input, positive_input, negative_input],
                      outputs=[encoded_anchor, encoded_positive, encoded_negative])

triplet_model_glot.compile(optimizer='adam', loss=triplet_loss)

In [44]:
train_num=500
test_num=100

In [77]:
triplets_train=create_train_triplets(X_train[0],y_train[0],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_spc.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1995
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2033
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2018
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2008
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2001
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2000
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2001
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.1998
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2005
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1962
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1928
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2009
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.2102
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2009
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - lo

In [78]:
triplets_test=create_test_triplets(X_test[0],y_test[0],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.035

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)

accuracy = (true_pos+false_pos) / test_num
true_pos = (true_pos) / total_pos
false_pos = (false_pos) / (test_num-total_pos)
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
0.021791425 : 1
0.054445446 : 0
0.02223116 : 0
0.05134139 : 0
0.0675305 : 0
0.024488667 : 0
0.08947568 : 0
0.022610929 : 1
0.047128014 : 0
0.03606747 : 0
0.034493685 : 0
0.03863159 : 1
0.036068518 : 0
0.032789253 : 0
0.017072544 : 0
0.024199478 : 0
0.01954997 : 0
0.016157482 : 0
0.05718883 : 1
0.033153553 : 0
0.08353829 : 0
0.045222055 : 0
0.050124153 : 0
0.031640086 : 0
0.047372106 : 0
0.035442658 : 0
0.055330206 : 1
0.044021398 : 0
0.07052455 : 0
0.09924775 : 0
0.04208876 : 0
0.016795682 : 0
0.027005425 : 1
0.06518319 : 0
0.020423649 : 0
0.020774882 : 0
0.02975196 : 0
0.07510645 : 0
0.03126304 : 0
0.027872939 : 0
0.025699807 : 0
0.014826084 : 0
0.028114202 : 0
0.022750277 : 1
0.057043713 : 0
0.03305138 : 0
0.069898 : 0
0.039130293 : 1
0.024776412 : 0
0.022769192 : 0
0.0415195 : 0
0.04990461 : 1
0.0387944 : 0
0.058033016 : 0
0.03157398 : 0
0.04565019 : 0
0.028149009 : 0
0.028359395 : 0
0.039539985 : 0
0.078084

In [79]:
triplets_train=create_train_triplets(X_train[1],y_train[1],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_vt.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.1998
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2002
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.1987
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2006
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.2105
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1966
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2204
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1986
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2010
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1990
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2020
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2006
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2012
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1988
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - lo

In [81]:
triplets_test=create_test_triplets(X_test[1],y_test[1],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.045

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)

accuracy = (true_pos+false_pos) / test_num
true_pos = (true_pos) / total_pos
false_pos = (false_pos) / (test_num-total_pos)
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
0.052947674 : 0
0.0778198 : 1
0.036589857 : 0
0.0561149 : 0
0.029159592 : 0
0.037801214 : 0
0.025132803 : 0
0.044609696 : 1
0.05985262 : 0
0.030084528 : 0
0.0 : 1
0.07531557 : 0
0.032109704 : 0
0.040209528 : 0
0.041274134 : 0
0.03876024 : 0
0.03508523 : 0
0.09961317 : 0
0.030577699 : 0
0.0752413 : 0
0.031005096 : 0
0.034701414 : 0
0.037509523 : 0
0.06984316 : 0
0.066586986 : 1
0.029987037 : 0
0.067363694 : 0
0.039104607 : 0
0.033959005 : 0
0.045044355 : 0
0.030328607 : 0
0.033006225 : 0
0.07173423 : 0
0.03249795 : 0
0.042249985 : 0
0.047092404 : 0
0.059109423 : 0
0.12456358 : 0
0.1230216 : 1
0.021556247 : 0
0.02624821 : 0
0.07319857 : 0
0.033286493 : 1
0.031793136 : 0
0.033315342 : 0
0.03626742 : 1
0.029926505 : 0
0.04691448 : 0
0.040648516 : 0
0.042145263 : 0
0.05415976 : 0
0.036713753 : 0
0.03012751 : 0
0.06332446 : 0
0.051348213 : 0
0.06591287 : 0
0.046069756 : 0
0.071023315 : 0
0.11103435 : 0
0.024571346 : 

In [82]:
triplets_train=create_train_triplets(X_train[2],y_train[2],train_num)

anchor_images = triplets_train[:, 0]
positive_images = triplets_train[:, 1]
negative_images = triplets_train[:, 2]

triplet_model_glot.fit([anchor_images, positive_images, negative_images], 
                  np.zeros((train_num, )),
                  batch_size=32, 
                  epochs=100)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.2013
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1992
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1991
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2090
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1999
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1982
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.1859
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1657
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.3341
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.1956
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1890
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1992
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2004
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.2014
Epoch 15/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - lo

In [86]:
triplets_test=create_test_triplets(X_test[2],y_test[2],test_num)

# anchor_images = triplets_test[:, 0]
# input_images = triplets_test[:, 1]
# label_test_spc = triplets_test[:, 2]

true_pos = 0
false_pos = 0

anchor_input_test = np.stack(triplets_test[:, 0])
positive_input_test = np.stack(triplets_test[:, 1])
labels_test = triplets_test[:, 2]

# print(np.sum(labels_test))

encoded_anchor = base_model.predict(anchor_input_test)
encoded_positive = base_model.predict(positive_input_test)

distances = np.linalg.norm(encoded_anchor - encoded_positive, axis=1)
threshold = 0.027

for i in range(test_num):
    print(str(distances[i])+" : "+str(labels_test[i]))
    if distances[i] < threshold and labels_test[i] == 1:
        true_pos += 1
    if distances[i] >= threshold and labels_test[i] == 0:
        false_pos += 1

total_pos=np.sum(labels_test)

accuracy = (true_pos+false_pos) / test_num
true_pos = (true_pos) / total_pos
false_pos  = (false_pos) / (test_num-total_pos)
print("True Positives: {:.2f}".format(true_pos))
print("False Positives: {:.2f}".format(false_pos))
print("Accuracy: {:.2f}".format(accuracy))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
0.026856562 : 0
0.038761158 : 0
0.025813947 : 0
0.03227409 : 0
0.03458691 : 0
0.014813623 : 0
0.03258713 : 0
0.02270225 : 0
0.027075818 : 0
0.033773642 : 0
0.023863131 : 1
0.03317374 : 0
0.025042534 : 0
0.026277672 : 0
0.021666192 : 0
0.017454566 : 0
0.019983392 : 1
0.025806218 : 0
0.02114003 : 0
0.022542208 : 0
0.031155676 : 0
0.03721855 : 0
0.01709048 : 0
0.03391986 : 0
0.022581544 : 0
0.030490462 : 0
0.03897039 : 0
0.032371126 : 0
0.018443527 : 0
0.015324241 : 0
0.023546677 : 0
0.016748495 : 0
0.027317317 : 0
0.04271736 : 1
0.019835634 : 0
0.029256953 : 0
0.016814828 : 0
0.034136154 : 0
0.023838205 : 0
0.03598133 : 1
0.023821112 : 0
0.049854215 : 0
0.04818937 : 0
0.03525801 : 0
0.036294498 : 0
0.02089801 : 0
0.024719257 : 0
0.02851094 : 1
0.032713532 : 0
0.03430833 : 0
0.028434906 : 0
0.0 : 1
0.020951165 : 0
0.04317774 : 0
0.026034001 : 0
0.033491675 : 0
0.038087975 : 0
0.023553425 : 0
0.024781102 : 0
0.0207

In [ ]:
# import zipfile

# source_path = '/kaggle/working'  
# zip_filename = 'saved_data.zip' 

# with zipfile.ZipFile(zip_filename, 'w') as zipf:

#     for root, _, files in os.walk(source_path):
#         for file in files:
#             zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), source_path))
